In [ ]:
import numpy as np
import pandas as pd
import pydicom as dicom
import matplotlib.pylab as plt
import glob
import SimpleITK as sitk
import re

import os
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
dicom.config.convert_wrong_length_to_UN = True

In [ ]:
#path = "F:/U-CAN-Lymfom_A/Ashish_SampleResampling/sample_data/"

#path = "F:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB"

source_path = "/media/andres/T7 Shield/ucan_lymfom"
destination_path = os.path.join(source_path, "metadata.xlsx")
selection_dataframe = os.path.join(source_path, "Final_Selected_exams_from_U-CAN-Lymfom.xlsx" )

In [ ]:
directory_list = list()
for root, dirs, files in os.walk(source_path, topdown=False):
    for name in dirs:
        directory_list.append(os.path.join(root, name))

In [ ]:
remove_list = [ 'PR----BONE-PULM-mm',
                'PR----Lunga-0.6-ax-mm',
                'PR----WB-Venfas-0.6-ax-mm',
                'PR----LUNG-1.25-AX-mm',
                'PR----WB-Ben-lunga-0.6-ax-mm',
                'PR----WB-Venfas-3-ax-mm',
                'PR----LUNG-1.25-AX-mm',
                'PR----BONE-1.25-AX-mm',
                'PR----LUNG-1.25-AX-mm',
                'PR----Lunga-0.6-ax-mm',
                'PR----SAVED-IMAGES-PR-mm',
                'PR----e1-QCFX-S-400-Static-mm',
                'PR----WB-Venfas-0.6-ax-mm',
                'PR----WB-VEN-AX-mm',
                'PR----WB-Ben-lunga-0.6-ax-mm',
                'PR----LUNG-1.25-AX-mm',
                'PR----THORAX-AX-mm',
                'PR----LUNG-1.25-AX-mm',
                'PR----THORAX-INANDAD-mm',
                'PR----KEY_IMAGES-PR-mm',
                'PR----SAVED-PR-mm',
                'Examinations that miss either CT or PET or both',
                'MR-',
                'sag',
                'cor',
                'ot-'
            ]

keep_list = ["CT-", "PT-"]

In [ ]:
findir_lst = []
rejection_lst = []

for dir in directory_list:
    dir = dir.replace('\\', '/')
    if any(item.lower() in dir.lower() for item in keep_list) and all(
            item.lower() not in dir.lower() for item in remove_list):
        print(dir)
        findir_lst.append(dir)
    else:
        rejection_lst.append(dir)

In [ ]:
len(rejection_lst)

In [ ]:
ucan_md_dict = {'dir': [],
                'source_dir': [],
                'patient_dir': [],
                'image_dir': [],
                'dicom_img': [],
                'patient_id': [],
                'patient_age': [],
                'patient_sex': [],
                'patient_weight': [],
                'patient_size': [],
                'rows': [],
                'columns': [],
                'num_slices': [],
                'pixel_spacing': [],
                'slice_thickness': [],
                'img_pos': [],
                'img_orient': [],
                'for_uid': [],
                'att_corr': [],
                'recons_method': [],
                'image_type': [],
                'aquisition_dt': [],
                'aquisition_time': [],
                'study_desc': [],
                'series_desc': [],
                'protocol': [],
                'corr_img': [],
                'modality': [],
                'manufacturer': [],
                'manufacturer_model': [],
                'radiopharmaceutical': [],
                'radiopharmaceutical_volume': [],
                'radiopharmaceutical_start_time': [],
                'radionuclide_total_dose': [],
                'radionuclide_half_life': [],
                'radionuclide_positron_fraction': [],
                'radiopharmaceutical_start_date_time': [],
                }

#emptydir_lst = []

for dir in findir_lst:
    #if len(os.listdir(dir))<100:
    #    emptydir_lst.append(dir)
    #    continue

    ucan_md_dict['dir'].append(dir)
    ucan_md_dict['dicom_img'].append(os.listdir(dir)[0])
    
    ucan_md_dict['source_dir'].append(dir.rsplit(sep='/', maxsplit=2)[-3])
    ucan_md_dict['patient_dir'].append(dir.rsplit(sep='/', maxsplit=2)[-2])
    ucan_md_dict['image_dir'].append(dir.rsplit(sep='/', maxsplit=2)[-1])  
                                            
    #print(os.listdir(dir)[0])
    ds = dicom.dcmread(dir + '/' + str(os.listdir(dir)[0]))

    meta_info = str(ds.fix_meta_info)#.split('\n')
    
    max_range = 500
    
    #PatientID
    try:
        id = re.search(r"\b(0010, 0020)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['patient_id'].append(element)
    except:
        ucan_md_dict['patient_id'].append('')
    
    #PatientAge 
    try:
        id = re.search(r"\b(0010, 1010)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['patient_age'].append(element)
    except:
        ucan_md_dict['patient_age'].append('')

    #PatientSex
    try:
        id = re.search(r"\b(0010, 0040)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['patient_sex'].append(element)
    except:
        ucan_md_dict['patient_sex'].append('')

    #Patient Size
    try:
        id = re.search(r"\b(0010, 1020)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['patient_size'].append(element)
    except:
        ucan_md_dict['patient_size'].append('')

    #Patient Weight
    try:
        id = re.search(r"\b(0010, 1030)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['patient_weight'].append(element)
    except:
        ucan_md_dict['patient_weight'].append('')
    
    #Rows
    try:
        id = re.search(r"\b(0028, 0010)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['rows'].append(element)
    except:
        ucan_md_dict['rows'].append('')

    #Columns
    try:
        id = re.search(r"\b(0028, 0011)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['columns'].append(element)
    except:
        ucan_md_dict['columns'].append('')

    #Num of slices
    try:
        id = re.search(r"\b(07a1, 1002)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['num_slices'].append(element)
    except:
        ucan_md_dict['num_slices'].append('')

    #Pixel spacing
    try:
        id = re.search(r"\b(0028, 0030)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['pixel_spacing'].append(element)
    except:
        ucan_md_dict['pixel_spacing'].append('')

    #Slice thickness
    try:
        id = re.search(r"\b(0018, 0050)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['slice_thickness'].append(element)
    except:
        ucan_md_dict['slice_thickness'].append('')

    #Image positioning
    try:
        id = re.search(r"\b(0020, 0032)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['img_pos'].append(element)
    except:
        ucan_md_dict['img_pos'].append('')

    #Image orientation
    try:
        id = re.search(r"\b(0020, 0037)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['img_orient'].append(element)
    except:
        ucan_md_dict['img_orient'].append('')

    #Frame of reference UID
    try:
        id = re.search(r"\b(0020, 0052)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['for_uid'].append(element)
    except:
        ucan_md_dict['for_uid'].append('')

    #Attenuation Correction Method
    try:
        id = re.search(r"\b(0054, 1101)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['att_corr'].append(element)
    except:
        ucan_md_dict['att_corr'].append('')

    #Reconstruction Method
    try:
        id = re.search(r"\b(0054, 1103)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['recons_method'].append(element)
    except:
        ucan_md_dict['recons_method'].append('')

    #Image Type
    try:
        id = re.search(r"\b(0008, 0008)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['image_type'].append(element)
    except:
        ucan_md_dict['image_type'].append('')

    #Aquisition Date
    try:
        id = re.search(r"\b(0008, 0022)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['aquisition_dt'].append(element)
    except:
        ucan_md_dict['aquisition_dt'].append('')
    
    #Aquisition Time
    try:
        id = re.search(r"\b(0008, 0032)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['aquisition_time'].append(element)
    except:
        ucan_md_dict['aquisition_time'].append('')

    #Study Description
    try:
        id = re.search(r"\b(0008, 1030)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['study_desc'].append(element)
    except:
        ucan_md_dict['study_desc'].append('')

    #Series Description
    try:
        id = re.search(r"\b(0008, 103e)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['series_desc'].append(element)
    except:
        ucan_md_dict['series_desc'].append('')

    #Protocol Name 
    try:
        id = re.search(r"\b(0018, 1030)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['protocol'].append(element)
    except:
        ucan_md_dict['protocol'].append('')

    #Corrected Image
    try:
        id = re.search(r"\b(0028, 0051)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['corr_img'].append(element)
    except:
        ucan_md_dict['corr_img'].append('')

    #Modality
    try:
        id = re.search(r"\b(0008, 0060)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['modality'].append(element)
    except:
        ucan_md_dict['modality'].append('')

    #Manufacturer
    try:
        id = re.search(r"\b(0008, 0070)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['manufacturer'].append(element)
    except:
        ucan_md_dict['manufacturer'].append('')

    #Manufacturer_model
    try:
        id = re.search(r"\b(0008, 1090)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['manufacturer_model'].append(element)
    except:
        ucan_md_dict['manufacturer_model'].append('')
    # Radiopharmaceutical
    try:
        id = re.search(r"\b(0018, 0031)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['radiopharmaceutical'].append(element)
    except:
        ucan_md_dict['radiopharmaceutical'].append('')
     # Radiopharmaceutical Volume
    try:
        id = re.search(r"\b(0018, 1071)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['radiopharmaceutical_volume'].append(element)
    except:
        ucan_md_dict['radiopharmaceutical_volume'].append('')
     # Radiopharmaceutical Start Time
    try:
        id = re.search(r"\b(0018, 1072)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['radiopharmaceutical_start_time'].append(element)
    except:
        ucan_md_dict['radiopharmaceutical_start_time'].append('')
     # Radionuclide Total Dose
    try:
        id = re.search(r"\b(0018, 1074)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['radionuclide_total_dose'].append(element)
    except:
        ucan_md_dict['radionuclide_total_dose'].append('')
     # Radionuclide Half Life
    try:
        id = re.search(r"\b(0018, 1075)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['radionuclide_half_life'].append(element)
    except:
        ucan_md_dict['radionuclide_half_life'].append('')
     # Radionuclide Positron Fraction
    try:
        id = re.search(r"\b(0018, 1076)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['radionuclide_positron_fraction'].append(element)
    except:
        ucan_md_dict['radionuclide_positron_fraction'].append('')
     # Radiopharmaceutical Start Date Time
    try:
        id = re.search(r"\b(0018, 1078)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['radiopharmaceutical_start_date_time'].append(element)
    except:
        ucan_md_dict['radiopharmaceutical_start_date_time'].append('')


In [ ]:
"""
with open(r'F:/U-CAN-Lymfom_A/Ashish_SampleResampling/metadata_extraction/dir_lessthan_100images.txt', 'w') as fp:
    for item in emptydir_lst:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

len(emptydir_lst)

Done
2399
"""

In [ ]:
ucan_md = pd.DataFrame(ucan_md_dict)
print(ucan_md.shape)
#(8325, 29)
#(8275, 29)
ucan_md.head(2)

In [ ]:
ucan_md.dtypes
ucan_md['imgsz_x'] = np.int64(ucan_md['rows'].str.replace("'","").replace(" ",""))
ucan_md['imgsz_y'] = np.int64(ucan_md['columns'])
ucan_md['num_slices'] = np.int64(ucan_md['num_slices'])
ucan_md['voxsz_x'] = ucan_md['pixel_spacing'].apply(lambda x: (np.float64(x.replace(" ","").replace("[","").replace("]","").split(',')[0])))
ucan_md['voxsz_y'] = ucan_md['pixel_spacing'].apply(lambda x: (np.float64(x.replace(" ","").replace("[","").replace("]","").split(',')[1])))
ucan_md['slice_thickness'] = np.float64(ucan_md['slice_thickness'].str.replace("'","").replace(" ",""))
ucan_md.head(2)

In [ ]:
ucan_md['image_size'] = ucan_md.apply(lambda x: (x.imgsz_x, x.imgsz_y, x.num_slices), axis=1)
ucan_md['voxel_size'] = ucan_md.apply(lambda x: (x.voxsz_x, x.voxsz_y, x.slice_thickness), axis=1)
ucan_md.head(2)

In [ ]:
ucan_md.columns

In [ ]:
#order columns
new_col_lst = ['dir', 'source_dir', 'patient_dir', 'image_dir', 'dicom_img',
       'patient_id', 'patient_age', 'patient_sex', 'patient_weight',
       'patient_size', 'rows', 'columns', 'imgsz_x', 'imgsz_y', 'num_slices', 
       'voxsz_x', 'voxsz_y', 'slice_thickness',  'pixel_spacing',
       'image_size', 'voxel_size', 'img_pos', 'img_orient', 'for_uid', 'att_corr',
       'recons_method', 'image_type', 'aquisition_dt', 'aquisition_time', 'study_desc',
       'series_desc', 'protocol', 'corr_img', 'modality', 'manufacturer',
       'manufacturer_model', 'radiopharmaceutical', 'radiopharmaceutical_volume',
       'radiopharmaceutical_start_time', 'radionuclide_total_dose', 
       'radionuclide_half_life', 'radionuclide_positron_fraction', 
       'radiopharmaceutical_start_date_time'
       ]

ucan_md = ucan_md[new_col_lst]

In [ ]:
print("Toal CT folders: ", ucan_md[ucan_md['image_dir'].str.contains('CT')].shape[0])
print("Toal PT folders: ", ucan_md[ucan_md['image_dir'].str.contains('PT')].shape[0])

In [ ]:
ucan_md.to_excel(destination_path)

In [ ]:
ucan_md.num_slices.min()

In [ ]:
ucan_md[ucan_md['num_slices']<=100].to_excel(os.path.join(source_path, 'exams_with_less_than_100_slices.xlsx'))

lessthan100slices_summ = ucan_md[np.int64(ucan_md['num_slices'])<=100].groupby(['modality', 'slice_thickness', 'num_slices']).agg({'dicom_img':'count'}).sort_values('dicom_img' ,ascending=False).reset_index()
lessthan100slices_summ.to_excel(os.path.join(source_path, 'exams_with_less_than_100_slices_final.xlsx'))

print(lessthan100slices_summ.shape)
#(785, 4)
lessthan100slices_summ.head(10)

In [ ]:
ucan_md_voxeldist = ucan_md.groupby(['modality', 'voxel_size']).agg({'dicom_img':'count'}).sort_values('dicom_img' ,ascending=False).reset_index()
ucan_md_voxeldist.to_excel(os.path.join(source_path, 'ucan_exams_md_voxeldist.xlsx'))
ucan_md_voxeldist.head(10)


In [ ]:
ucan_md_imgszdist = ucan_md.groupby(['modality', 'image_size']).agg({'dicom_img':'count'}).sort_values('dicom_img' ,ascending=False).reset_index()
ucan_md_imgszdist.to_excel(os.path.join(source_path, 'ucan_exams_md_imgszdist.xlsx'))
ucan_md_imgszdist.head(10)

In [ ]:
ucan_md_gt200slices = ucan_md[ucan_md['num_slices']>=200]
ucan_md_gt200slices.to_excel(os.path.join(source_path, 'ucan_exams_with_more_than_200_slices.xlsx'))
print(ucan_md_gt200slices.shape)
ucan_md_gt200slices.head(2)

In [ ]:
ucan_md_gt200slices.groupby('voxel_size').agg({'dicom_img':'count'}).sort_values('dicom_img' ,ascending=False)

In [ ]:
ucan_md_gt200slices_voxeldist = ucan_md_gt200slices.groupby(['modality', 'voxel_size']).agg({'dicom_img':'count'}).sort_values('dicom_img' ,ascending=False).reset_index()
ucan_md_gt200slices_voxeldist.to_excel(os.path.join(source_path, 'ucan_exams_with_more_than_200_slices_voxeldist.xlsx'))
ucan_md_gt200slices_voxeldist.head(10)


In [ ]:
ucan_md_gt200slices_imgszdist = ucan_md_gt200slices.groupby(['modality', 'image_size']).agg({'dicom_img':'count'}).sort_values('dicom_img' ,ascending=False).reset_index()
ucan_md_gt200slices_imgszdist.to_excel(os.path.join(source_path, 'ucan_exams_with_more_than_200_slices_imagesizedist.xlsx'))
ucan_md_gt200slices_imgszdist.head(10)

In [ ]:
Lessthan_200Slices = ucan_md[ucan_md['num_slices']<200]
Lessthan_200Slices.to_excel(os.path.join(source_path, "less_than_200_slices.xlsx"))
print(Lessthan_200Slices.shape)
Lessthan_200Slices.head(2)

In [ ]:
ucan_md[ucan_md['image_dir'].str.contains('-ax-')][ucan_md['num_slices']<200][ucan_md['image_dir'].str.contains('PT-')]

In [ ]:
ucan_md[ucan_md['num_slices']<200]

In [ ]:
ucan_md.groupby(['patient_sex']).sum()

### Match metadata with selected files

In [ ]:
metadata = pd.read_excel(destination_path)
# metadata['dir'] = metadata['dir'].str.replace('/','\\')
print(metadata.shape)
metadata.head(2)

In [ ]:
selected_imgs = pd.read_excel(selection_dataframe)
print(selected_imgs.shape)
selected_imgs.head(2)

In [ ]:
master_data = pd.merge(selected_imgs, metadata, how="inner", left_on=['directory'], right_on=['dir'], sort=True, suffixes=("_x", "_y"))

col_drop_lst = ['dir', 'source_dir', 'patient_dir',	'image_dir', 'patient_sex', 'rows',	'columns', 'pixel_spacing']

master_data = master_data.drop(columns=col_drop_lst)

master_data['patient_id'] = master_data['patient_id'].apply(lambda x: x.split('_')[1].replace("'",""))
master_data['patient_age'] = master_data['patient_age'].apply(lambda x: x.replace("'","").replace("Y","").replace(" ",""))#.astype('Int64')
master_data['modality'] = master_data['modality'].apply(lambda x: x.strip().replace("'","").strip())

master_data.to_excel(os.path.join(source_path, "Finalized_dataset.xlsx"))

print(master_data.shape)
master_data.head(2)

In [ ]:
print(master_data[master_data['patient_age']==''].shape)

master_data[master_data['patient_age']=='']

In [ ]:
master_data_imgszdist = master_data.groupby(['modality', 'image_size']).agg({'dicom_img':'count'}).sort_values('dicom_img' ,ascending=False).reset_index()
master_data_imgszdist.to_excel(os.path.join(source_path, "master_data_imgszdist.xlsx"))
print(master_data_imgszdist.shape)
master_data_imgszdist.head(10)

In [ ]:
master_data_voxszdist = master_data.groupby(['modality', 'voxel_size']).agg({'dicom_img':'count'}).sort_values('dicom_img' ,ascending=False).reset_index()
master_data_voxszdist.to_excel(os.path.join(source_path, "master_data_voxszdist.xlsx"))
print(master_data_voxszdist.shape)
master_data_voxszdist.head(10)

In [ ]:
master_data.columns

In [ ]:
master_data = pd.read_excel(os.path.join(source_path, "Finalized_dataset.xlsx"))

#master_data['modality'] = master_data['modality'].apply(lambda x: x.strip().replace("'","").strip())
#master_data.to_csv('F:/U-CAN-Lymfom_A/Selected_for_UCAN_project/Selected_exams_from_U-CAN-Lymfom_A_MasterDataset_2Oct2023.csv', index=False)


In [ ]:
print("Mode of x image dims: ", master_data['imgsz_x'].mode()[0])
print("Mode of y image dims: ", master_data['imgsz_y'].mode()[0])
print("Mode of z image dims: ", master_data['num_slices'].mode()[0])
print("Mode of x voxel dims: ", master_data['voxsz_x'].mode()[0])
print("Mode of y voxel dims: ", master_data['voxsz_y'].mode()[0])
print("Mode of z voxel dims: ", master_data['slice_thickness'].mode()[0])

In [ ]:
master_data[['imgsz_x','imgsz_y','num_slices','voxsz_x','voxsz_y','slice_thickness']].describe()

### New size and spacing based on master data

new_size = [384, 384, 384]
new_spacing = [2, 2, 3]

In [ ]:
master_data.head(2)
master_data.npr.nunique()

In [ ]:
master_data.patient_directory.nunique()